In [3]:
import os
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [4]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

In [5]:
from modules.dataset import Dataset
from modules.models import Model
from modules.parser import parse_option

In [6]:
weight_option = None # use `imagenet` or `None` only

In [7]:
# Constant variables
NAME = "EfficientNetB0"
EPOCHS = 100
NUM_FOLDS = 5

In [8]:
# Learning rate
def lr_schedule(epoch, learning_rate):
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [9]:
def get_callbacks(NAME, weight_option, fold_num):
    model_checkpoint_callback = ModelCheckpoint(f'results/models/facal_loss/{NAME}_{weight_option}_fold_{fold_num}.h5', monitor='val_loss', mode='min', save_best_only=True)
    early_stop_callback = EarlyStopping(monitor='val_loss', mode="min", patience=20, verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.5, patience=3, verbose=1)
    lr_logging_callback = LearningRateScheduler(lr_schedule)
    
    return model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback
    

In [10]:
dataset = Dataset()

In [11]:
fold_num = 3 # use values [1-5]

In [13]:
# Callbacks
model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback = get_callbacks(NAME, weight_option, fold_num)

# Path for CSV
path = os.path.join("results", "history", "training_with_facal_loss", f"{NAME}_{weight_option}")
os.makedirs(path, exist_ok=True)

# CSV Logger
csv_logger = CSVLogger(os.path.join(path, f"fold_{fold_num}.csv"))

# Dataset
train_dataset, test_dataset = dataset.get_kfold(fold_num, sample=False)

# Modeling
transfer_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False, 
    weights=weight_option,
    input_shape=(224, 224, 3),
    pooling=None
)

loss_function = tf.keras.losses.BinaryFocalCrossentropy(
    # apply_class_balancing=True,
    from_logits=True,
)

model = Model(
    transfer_model,
    loss_function
)
model = model.get_model()
model.summary()

# Visualize
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    verbose=1, # Show Progress Bar while Traning
    callbacks=[model_checkpoint_callback, csv_logger, early_stop_callback, reduce_lr_callback, lr_logging_callback]
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten_1 (Flatten)         (None, 62720)             0         
                                                                 
 dense_5 (Dense)             (None, 128)               8028288   
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 15)               

## Try drop out

In [17]:
""" Documations
a module for create a model easier for experimental
"""

import tensorflow as tf

class Model:
    """ Use for create a different transfer learning model
    Example
    --------
    >>> from modules.models import Model
    >>> transfer_model = tf.keras.applications.resnet50.ResNet50(
        include_top=True, 
        weights=None,
        pooling='avg'
    )
    >>> model = Model(transfer_model).get_model()
    """
    def __init__(self, transfer_model, loss_function):
        self.optimizer = tf.keras.optimizers.Adam()
        self.loss = loss_function
        self.metrics = [tf.keras.metrics.AUC(multi_label=True)]
        self.transfer = transfer_model
        self.model = None
        
    def create_model(self):
        """ Create a Sequential of model for instances
        Example
        -------
        >>> self.create_model()
        """        
        sequential_list = [self.transfer]
        sequential_list.append(tf.keras.layers.Flatten())
        
        sequential_list = sequential_list + [
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dropout(0.2),
            
            tf.keras.layers.Dense(15, activation='sigmoid')
        ]
        self.model = tf.keras.Sequential(sequential_list)
    
    def compile_model(self):
        """Compile a Sequential of model for instances
        Example
        -------
        >>> self.compile_model()
        """        
        self.model.compile(optimizer=self.optimizer,
                           loss=self.loss,
                           metrics=self.metrics)
        
    def get_model(self):
        """Create and Compile a Sequential of model for instances

        Returns
        -------
        tf.Model
            a Sequential of model
        """
        self.create_model()
        self.compile_model()
        return self.model

In [18]:
def get_callbacks(NAME, weight_option, fold_num, path="/home/jovyan/ChestXray-14/run_specific_fold"):
    model_checkpoint_callback = ModelCheckpoint(f'{path}/{NAME}_{weight_option}_fold_{fold_num}.h5', monitor='val_loss', mode='min', save_best_only=True)
    early_stop_callback = EarlyStopping(monitor='val_loss', mode="min", patience=20, verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.5, patience=3, verbose=1)
    lr_logging_callback = LearningRateScheduler(lr_schedule)
    
    return model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback

In [19]:
# Path
path = os.path.join("results", "history", "training_with_facal_loss", "dropout_20", f"{NAME}_{weight_option}")
os.makedirs(path, exist_ok=True)

# Callbacks
model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback = get_callbacks(
    NAME, weight_option, fold_num, path=path
)

# CSV Logger
csv_logger = CSVLogger(os.path.join(path, f"fold_{fold_num}.csv"))

# Dataset
train_dataset, test_dataset = dataset.get_kfold(fold_num, sample=False)

# Modeling
transfer_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False, 
    weights=weight_option,
    input_shape=(224, 224, 3),
    pooling=None
)

loss_function = tf.keras.losses.BinaryFocalCrossentropy(
    # apply_class_balancing=True,
    from_logits=True,
)

model = Model(
    transfer_model,
    loss_function
)
model = model.get_model()
model.summary()

# Visualize
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    verbose=1, # Show Progress Bar while Traning
    callbacks=[model_checkpoint_callback, csv_logger, early_stop_callback, reduce_lr_callback, lr_logging_callback]
)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten_2 (Flatten)         (None, 62720)             0         
                                                                 
 dense_10 (Dense)            (None, 128)               8028288   
                                                                 
 dense_11 (Dense)            (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 64)               